Total Game Score Model - Model Tuning and Building - GBM

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')
from total_points_model.domain.preprocessing.data_preprocessor import DataPreprocessor
from total_points_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from total_points_model.domain.modelling.supermodel import SuperXGBRegressor

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Load Data

In [2]:
afl_data = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/total-points-score-model/data/merged-data/afl_match_data.csv")

In [3]:
afl_data['ModellingFilter'] = np.where(afl_data['Date'] < "2019-01-01", True, False)
afl_data = afl_data[afl_data['Year'] >= 2010]

afl_data = afl_data.rename(columns = {'Total Game Score':'Total_Game_Score'})

In [11]:
training_data = afl_data[afl_data['ModellingFilter']]
test_data = afl_data[~afl_data['ModellingFilter']]

In [12]:
response = "Total_Game_Score"

In [13]:
X_train, y_train = training_data.drop(columns = [response]), training_data[response]
X_test, y_test = test_data.drop(columns = [response]), test_data[response]

Preprocess Data

In [14]:
afl_rename_dict = {
    'Total Game Score':'Total_Game_Score',
    'Home Win':'Home_Win'
    }

In [15]:
preprocessor = DataPreprocessor(rename_dict=afl_rename_dict)

In [16]:
preprocessor.fit(X_train)

DataPreprocessor(rename_dict={'Home Win': 'Home_Win',
                              'Total Game Score': 'Total_Game_Score'})

In [17]:
X_train_preproc = preprocessor.transform(X_train)
X_test_preproc = preprocessor.transform(X_test)

In [18]:
X_train_preproc.head()

,Year,Temperature,random5,Home_Total_Q1_Score_avg2,Home_Total_Q1_Goals_avg2,Home_Total_Q1_Behinds_avg2,Home_Total_Q1_Shots_avg2,Home_Total_Q1_Conversion_avg2,Home_Total_Q2_Score_avg2,Home_Total_Q2_Goals_avg2,Home_Total_Q2_Behinds_avg2,Home_Total_Q2_Shots_avg2,Home_Total_Q2_Conversion_avg2,Home_Total_Q3_Score_avg2,Home_Total_Q3_Goals_avg2,Home_Total_Q3_Behinds_avg2,Home_Total_Q3_Shots_avg2,Home_Total_Q3_Conversion_avg2,Home_Total_Q4_Score_avg2,Home_Total_Q4_Goals_avg2,Home_Total_Q4_Behinds_avg2,Home_Total_Q4_Shots_avg2,Home_Total_Q4_Conversion_avg2,Home_Att_Q1_Score_avg2,Home_Att_Q1_Goals_avg2,Home_Att_Q1_Behinds_avg2,Home_Att_Q1_Shots_avg2,Home_Att_Q1_Conversion_avg2,Home_Att_Q2_Score_avg2,Home_Att_Q2_Goals_avg2,Home_Att_Q2_Behinds_avg2,Home_Att_Q2_Shots_avg2,Home_Att_Q2_Conversion_avg2,Home_Att_Q3_Score_avg2,Home_Att_Q3_Goals_avg2,Home_Att_Q3_Behinds_avg2,Home_Att_Q3_Shots_avg2,Home_Att_Q3_Conversion_avg2,Home_Att_Q4_Score_avg2,Home_Att_Q4_Goals_avg2,Home_Att_Q4_Behinds_avg2,Home_Att_Q4_Shots_avg2,Home_Att_Q4_Conversion_avg2,Home_Def_Q1_Score_avg2,Home_Def_Q1_Goals_avg2,Home_Def_Q1_Behinds_avg2,Home_Def_Q1_Shots_avg2,Home_Def_Q1_Conversion_avg2,Home_Def_Q2_Score_avg2,Home_Def_Q2_Goals_avg2,Home_Def_Q2_Behinds_avg2,Home_Def_Q2_Shots_avg2,Home_Def_Q2_Conversion_avg2,Home_Def_Q3_Score_avg2,Home_Def_Q3_Goals_avg2,Home_Def_Q3_Behinds_avg2,Home_Def_Q3_Shots_avg2,Home_Def_Q3_Conversion_avg2,Home_Def_Q4_Score_avg2,Home_Def_Q4_Goals_avg2,Home_Def_Q4_Behinds_avg2,Home_Def_Q4_Shots_avg2,Home_Def_Q4_Conversion_avg2,Away_Total_Q1_Score_avg2,Away_Total_Q1_Goals_avg2,Away_Total_Q1_Behinds_avg2,Away_Total_Q1_Shots_avg2,Away_Total_Q1_Conversion_avg2,Away_Total_Q2_Score_avg2,Away_Total_Q2_Goals_avg2,Away_Total_Q2_Behinds_avg2,Away_Total_Q2_Shots_avg2,Away_Total_Q2_Conversion_avg2,Away_Total_Q3_Score_avg2,Away_Total_Q3_Goals_avg2,Away_Total_Q3_Behinds_avg2,Away_Total_Q3_Shots_avg2,Away_Total_Q3_Conversion_avg2,Away_Total_Q4_Score_avg2,Away_Total_Q4_Goals_avg2,Away_Total_Q4_Behinds_avg2,Away_Total_Q4_Shots_avg2,Away_Total_Q4_Conversion_avg2,Away_Att_Q1_Score_avg2,Away_Att_Q1_Goals_avg2,Away_Att_Q1_Behinds_avg2,Away_Att_Q1_Shots_avg2,Away_Att_Q1_Conversion_avg2,Away_Att_Q2_Score_avg2,Away_Att_Q2_Goals_avg2,Away_Att_Q2_Behinds_avg2,Away_Att_Q2_Shots_avg2,Away_Att_Q2_Conversion_avg2,Away_Att_Q3_Score_avg2,Away_Att_Q3_Goals_avg2,Away_Att_Q3_Behinds_avg2,Away_Att_Q3_Shots_avg2,Away_Att_Q3_Conversion_avg2,Away_Att_Q4_Score_avg2,Away_Att_Q4_Goals_avg2,Away_Att_Q4_Behinds_avg2,Away_Att_Q4_Shots_avg2,Away_Att_Q4_Conversion_avg2,Away_Def_Q1_Score_avg2,Away_Def_Q1_Goals_avg2,Away_Def_Q1_Behinds_avg2,Away_Def_Q1_Shots_avg2,Away_Def_Q1_Conversion_avg2,Away_Def_Q2_Score_avg2,Away_Def_Q2_Goals_avg2,Away_Def_Q2_Behinds_avg2,Away_Def_Q2_Shots_avg2,Away_Def_Q2_Conversion_avg2,Away_Def_Q3_Score_avg2,Away_Def_Q3_Goals_avg2,Away_Def_Q3_Behinds_avg2,Away_Def_Q3_Shots_avg2,Away_Def_Q3_Conversion_avg2,Away_Def_Q4_Score_avg2,Away_Def_Q4_Goals_avg2,Away_Def_Q4_Behinds_avg2,Away_Def_Q4_Shots_avg2,Away_Def_Q4_Conversion_avg2,Home_Team_Adelaide,Home_Team_Brisbane Lions,Home_Team_Carlton,Home_Team_Collingwood,Home_Team_Essendon,Home_Team_Fremantle,Home_Team_Geelong,Home_Team_Gold Coast,Home_Team_Greater Western Sydney,Home_Team_Hawthorn,Home_Team_Melbourne,Home_Team_North Melbourne,Home_Team_Port Adelaide,Home_Team_Richmond,Home_Team_St Kilda,Home_Team_Sydney,Home_Team_West Coast,Home_Team_Western Bulldogs,Away_Team_Adelaide,Away_Team_Brisbane Lions,Away_Team_Carlton,Away_Team_Collingwood,Away_Team_Essendon,Away_Team_Fremantle,Away_Team_Geelong,Away_Team_Gold Coast,Away_Team_Greater Western Sydney,Away_Team_Hawthorn,Away_Team_Melbourne,Away_Team_North Melbourne,Away_Team_Port Adelaide,Away_Team_Richmond,Away_Team_St Kilda,Away_Team_Sydney,Away_Team_West Coast,Away_Team_Western Bulldogs,Venue_Adelaide Oval,Venue_Bellerive Oval,Venue_Carrara,Venue_Cazalys Stadium,Venue_Docklands,Venue_Eureka Stadium,Venue_Football Park,Venue_Gabba,Venue_Kardinia Park,Venue_M.C.G.,Venue_Manuka 

In [19]:
X_train_preproc.shape, X_test_preproc.shape

((1830, 449), (783, 449))

Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [23]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train)

In [24]:
xgb_tuner.tune_hyperparameters()

[I 2023-05-13 14:37:21,026] A new study created in memory with name: no-name-525ef16a-d5f2-4433-be4b-12603d9b6661
[I 2023-05-13 14:37:21,088] Trial 0 finished with value: 19.143490651298944 and parameters: {'max_depth': 2, 'min_child_weight': 12, 'eta': 0.5790638387741711, 'gamma': 20.0, 'lambda': 0.019008070897790257, 'alpha': 0.0026253652272019323, 'subsample': 0.6822808635539024, 'colsample_bytree': 0.7583629622667742}. Best is trial 0 with value: 19.143490651298944.
[I 2023-05-13 14:37:21,164] Trial 1 finished with value: 164.0276100947164 and parameters: {'max_depth': 8, 'min_child_weight': 2, 'eta': 0.009098947486020387, 'gamma': 20.0, 'lambda': 0.00038641538754507197, 'alpha': 0.008791623826307278, 'subsample': 0.2350983067369099, 'colsample_bytree': 0.2098550200124925}. Best is trial 0 with value: 19.143490651298944.
[I 2023-05-13 14:37:21,290] Trial 2 finished with value: 181.57564668246349 and parameters: {'max_depth': 16, 'min_child_weight': 7, 'eta': 0.00012104885706505487,

Number of finished trials:  1000
Best trial:
  Value: 17.271392527712106
  Params: 
    max_depth: 4
    min_child_weight: 12
    eta: 0.38592781828256695
    gamma: 20.0
    lambda: 0.07466041865676804
    alpha: 0.8209737237011836
    subsample: 0.8469878093771339
    colsample_bytree: 0.605768880041442


In [25]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 4,
 'min_child_weight': 12,
 'eta': 0.38592781828256695,
 'gamma': 20.0,
 'lambda': 0.07466041865676804,
 'alpha': 0.8209737237011836,
 'subsample': 0.8469878093771339,
 'colsample_bytree': 0.605768880041442}

Training Model - SuperXGBRegressor class for training and predictions

In [26]:
params['objective'] = 'reg:squarederror'
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1

In [27]:
super_xgb = SuperXGBRegressor(X_train = X_train_preproc, 
                              y_train = y_train, 
                              X_test = X_test_preproc, 
                              y_test = y_test, 
                              params = params)

In [28]:
super_xgb.fit()

[0]	validation_0-rmse:111.79562	validation_1-rmse:96.30232
[1]	validation_0-rmse:70.19371	validation_1-rmse:61.30941
[2]	validation_0-rmse:45.26629	validation_1-rmse:40.56266
[3]	validation_0-rmse:30.80099	validation_1-rmse:28.95227
[4]	validation_0-rmse:22.96138	validation_1-rmse:22.96689
[5]	validation_0-rmse:18.81473	validation_1-rmse:20.24269
[6]	validation_0-rmse:16.88731	validation_1-rmse:18.92090
[7]	validation_0-rmse:15.88461	validation_1-rmse:18.33178
[8]	validation_0-rmse:15.36235	validation_1-rmse:18.08680
[9]	validation_0-rmse:15.05910	validation_1-rmse:17.98987
[10]	validation_0-rmse:14.84365	validation_1-rmse:17.96341
[11]	validation_0-rmse:14.67670	validation_1-rmse:17.99800
[12]	validation_0-rmse:14.41815	validation_1-rmse:18.04722
[13]	validation_0-rmse:14.30247	validation_1-rmse:18.04137
[14]	validation_0-rmse:14.14099	validation_1-rmse:18.03438
[15]	validation_0-rmse:14.03390	validation_1-rmse:18.07707
[16]	validation_0-rmse:13.72153	validation_1-rmse:18.11525
[17]	v

In [29]:
super_xgb.xgb_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.605768880041442, early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=20.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.38592781828256695,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=12, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [30]:
test_preds = super_xgb.predict(X_test_preproc)

Export model

In [31]:
super_xgb.export_model("/Users/ciaran/Documents/Projects/AFL/total-points-score-model/afl-total-points-model/total_points_model/models/xgb_regression_test_v1.joblib")